# Optimasi Portofolio Non-Linear dengan Pertimbangan Risiko Tingkat Tinggi dan Biaya Transaksi Nonlinier pada Saham Perbankan Indonesia

## Scraping

In [ ]:
#Scraping data
tickers = ['BBCA.JK', 'BBRI.JK', 'BMRI.JK', 'BBNI.JK']

#2022-01-01 sampai 2025-10-31
start = '2022-01-01'
end = '2025-10-31'

#Ambil harga penutupan
data = yf.download(tickers, start=start, end=end)['Close']
data.head()
data.to_csv('data_saham_bank.csv')

/tmp/ipython-input-556454887.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start, end=end)['Close']
[*********************100%***********************]  4 of 4 completed


Untuk analisis portofolio saham perbankan Indonesia periode 2022-2025, dilakukan scraping data harga penutupan harian dari Yahoo Finance menggunakan pustaka yfinance. Hanya harga penutupan (Close) yang diambil untuk keperluan perhitungan return.

## 3a. Formulasi Ulang Model dalam bentuk Matematis



### Formulasi Model Portofolio Nonlinear (NLP)

#### Fungsi Objektif

$$
\begin{aligned}
\text{Maksimalkan:} \quad U(w) =\ &\mu^T w
\ -\ \lambda_1\, w^T \Sigma w \\
&+\ \lambda_2 \sum_{i=1}^4 \text{skew}_i \, w_i^3 \\
&-\ \lambda_3 \sum_{i=1}^4 \text{kurt}_i \, w_i^4 \\
&-\ \sum_{i=1}^4 \left[ a_i (w_i - w_{0,i})^2 + b_i \sqrt{(w_i - w_{0,i})^2 + \epsilon} \right]
\end{aligned}
$$

#### Kendala (Constraints)

1. **Budget Constraint (Total Investasi):**
   $$
   \sum_{i=1}^4 w_i = 1
   $$

2. **Batas Individu Saham:**
   $$
   -0.2 \leq w_i \leq 0.6, \quad i = 1,2,3,4
   $$

3. **Leverage Constraint:**
   $$
   \sum_{i=1}^4 \sqrt{w_i^2 + \epsilon} \leq 1.5
   $$

4. **Turnover Constraint (Total transaksi dari porto awal):**
   $$
   \sum_{i=1}^4 \sqrt{(w_i - w_{0,i})^2 + \epsilon} \leq 0.6
   $$

5. **Sharpe Ratio Minimum (minimal rasio return):**
   $$
   \frac{\mu^T w}{\sqrt{w^T \Sigma w}} \geq 0.25
   $$

#### Definisi Variabel
$w_i$ : bobot proporsi portofolio pada saham ke-$i$, untuk $i = 1, \ldots, 4$  
$\mu$ : vektor return ekspektasi tahunan $(4 \times 1)$  
$\Sigma$ : matriks kovarians return tahunan $(4 \times 4)$  
$\mathrm{skew}_i$, $\mathrm{kurt}_i$ : skewness dan kurtosis untuk setiap saham $i$  
$w_0$ : bobot portofolio awal (misal, 0.25)  
$a_i$, $b_i$ : parameter biaya transaksi untuk saham $i$  
$\lambda_1$, $\lambda_2$, $\lambda_3$ : parameter preferensi risiko/asimetri  
$\epsilon$ : konstanta kecil untuk stabilitas numerik



## 3b. Implementasi Model

In [ ]:
df = pd.read_csv("data_saham_bank.csv", index_col=0, parse_dates=True)
df.head()

,BBCA.JK,BBNI.JK,BBRI.JK,BMRI.JK
Date,,,,
2022-01-03,6616.546387,2772.385254,3284.902100,2767.065186
2022-01-04,6684.291504,2896.060303,3269.185303,2816.126709
2022-01-05,6729.456055,2865.141846,3308.478027,2757.252930
2022-01-06,6752.039062,2865.141846,3269.185303,2757.252930
2022-01-07,6910.112793,2916.672852,3292.760986,2767.065186


In [ ]:
order = ['BBCA.JK', 'BBNI.JK', 'BBRI.JK', 'BMRI.JK']
df = df[order]

#Hitung return
returns = df.pct_change().dropna()

#Parameter harian
mu_daily = returns.mean().values                # vektor ekspektasi return
Sigma_daily = returns.cov().values              # matriks kovarians
skew = returns.skew().values              # koefisien skewness
kurt = returns.kurtosis().values          # koefisien kurtosis


#ANNUALISASI (scaling ke tahunan)
TRADING_DAYS = 252
mu = mu_daily * TRADING_DAYS
Sigma = Sigma_daily * TRADING_DAYS

# Print statistik untuk verifikasi
print("="*70)
print("STATISTIK PARAMETER (Annualized)")
print("="*70)
summary = pd.DataFrame({
    'mean_daily': mu_daily,
    'mean_annual': mu,
    'skew': skew,
    'kurtosis': kurt
}, index=order)
print(summary)
print("\nAnnualized Covariance Matrix:")
print(pd.DataFrame(Sigma, index=order, columns=order))
print("="*70 + "\n")

STATISTIK PARAMETER (Annualized)
         mean_daily  mean_annual      skew  kurtosis
BBCA.JK    0.000385     0.096909  0.146054  2.752057
BBNI.JK    0.000686     0.172943  0.262492  2.273141
BBRI.JK    0.000355     0.089536  0.122579  2.898439
BMRI.JK    0.000781     0.196797  0.050991  3.178258

Annualized Covariance Matrix:
          BBCA.JK   BBNI.JK   BBRI.JK   BMRI.JK
BBCA.JK  0.054485  0.035709  0.036006  0.038481
BBNI.JK  0.035709  0.090812  0.055262  0.056829
BBRI.JK  0.036006  0.055262  0.083384  0.052199
BMRI.JK  0.038481  0.056829  0.052199  0.090562



In [ ]:
w0 = np.array([0.25, 0.25, 0.25, 0.25])  # Bobot portfolio awal
a = np.full(4, 10.0)                      # Parameter transaction cost (quadratic)
b = np.full(4, 0.5)
eps = 1e-8
lambda1, lambda2, lambda3 = 1.0, 0.2, 0.1
N = 4
idx = range(N)


#Optimisasi multistart
results_list = []
n_start = 10

print(f"Running {n_start} optimization trials with different initializations...\n")

for k in range(n_start):
    model = pyo.ConcreteModel()

    #Inisialisasi random
    rand_init = np.random.uniform(-0.15, 0.55, size=N)
    model.w = pyo.Var(idx, domain=pyo.Reals, bounds=(-0.2, 0.6),
                      initialize={i: rand_init[i] for i in idx})

    #Fungsi objektif (utility maximization)
    def utility(m):
        wvec = np.array([m.w[i]() for i in idx])
        ret = np.dot(mu, wvec)
        risk = np.dot(wvec, np.dot(Sigma, wvec))
        skew_term = np.sum(skew * wvec**3)
        kurt_term = np.sum(kurt * wvec**4)
        trans_cost = np.sum(a * (wvec - w0)**2 + b * np.sqrt((wvec - w0)**2 + eps))
        util = ret - lambda1 * risk + lambda2 * skew_term - lambda3 * kurt_term - trans_cost
        return -util

    model.utility = pyo.Objective(rule=utility, sense=pyo.minimize)

    #Constraint 1: Budget (sum to 1)
    model.budget = pyo.Constraint(expr=sum(model.w[i] for i in idx) == 1)

    #Constraint 2: Leverage limit
    model.leverage = pyo.Constraint(
        expr=sum(pyo.sqrt(model.w[i]**2 + eps) for i in idx) <= 1.5
    )

    # onstraint 3: Turnover limit
    model.turnover = pyo.Constraint(
        expr=sum(pyo.sqrt((model.w[i] - w0[i])**2 + eps) for i in idx) <= 0.6
    )

    #Constraint 4: Sharpe ratio minimum
    def sharpe_rule(m):
        ret = sum(mu[i] * m.w[i] for i in idx)
        risk = sum(m.w[i] * sum(Sigma[i][j] * m.w[j] for j in idx) for i in idx)
        return ret >= 0.25 * pyo.sqrt(risk + eps)

    model.sharpe = pyo.Constraint(rule=sharpe_rule)

    #Solver
    SOLVER = pyo.SolverFactory('ipopt', executable='/content/bin/ipopt')
    out = SOLVER.solve(model, tee=False)

    #Extract solusi jika optimal
    if (out.solver.status == pyo.SolverStatus.ok) and \
       (out.solver.termination_condition == pyo.TerminationCondition.optimal):
        w_star = np.array([pyo.value(model.w[i]) for i in idx])
        U_star = -pyo.value(model.utility)
        results_list.append(dict(seed=k, w_star=w_star, U_star=U_star))
        print(f"Trial {k+1}: Converged (U = {U_star:.6f})")
    else:
        print(f"Trial {k+1}: Infeasible")

Running 10 optimization trials with different initializations...

Trial 1: Converged (U = -1.519961)
Trial 2: Converged (U = -2.898969)
Trial 3: Converged (U = -1.990213)
Trial 4: Converged (U = -2.982661)
Trial 5: Converged (U = -0.451625)
Trial 6: Converged (U = -1.266143)
Trial 7: Converged (U = -0.698764)
Trial 8: Converged (U = -2.448460)
Trial 9: Converged (U = -1.737447)
Trial 10: Converged (U = -1.993420)


In [ ]:
print("HASIL OPTIMISASI")
print("="*70)

if results_list:
    best = max(results_list, key=lambda x: x['U_star'])
    w_best = best['w_star']

    print(f"\nBest solution found from {len(results_list)} successful trials:")
    print("-" * 70)
    print("Optimal Portfolio Weights:")
    for i, ticker in enumerate(order):
        print(f"  {ticker}: {w_best[i]:.4f}")
    print(f"\nMaximum Utility: {best['U_star']:.8f}")
    print("-" * 70)

    all_utils = [float(r['U_star']) for r in results_list]
    print(f"\nAll utility values: {all_utils}")

    unique_utils = len(set([round(u, 8) for u in all_utils]))
    if unique_utils == 1:
        print("\n All runs converge to the same solution, solusi adalah GLOBAL optimum")
    else:
        print(f"\n Found {unique_utils} different solutions, solusi adalah LOCAL optimum")

HASIL OPTIMISASI

Best solution found from 10 successful trials:
----------------------------------------------------------------------
Optimal Portfolio Weights:
  BBCA.JK: 0.2346
  BBNI.JK: 0.2857
  BBRI.JK: 0.2058
  BMRI.JK: 0.2739

Maximum Utility: -0.45162485
----------------------------------------------------------------------

All utility values: [-1.5199606062128823, -2.898969395407835, -1.9902134270868819, -2.9826605092994543, -0.45162485185510254, -1.2661427305090038, -0.6987637121145839, -2.448460047811889, -1.7374474938880629, -1.9934195091644247]

 Found 10 different solutions, solusi adalah LOCAL optimum


Hasil optimasi portofolio menunjukkan bahwa dari 10 kali percobaan dengan inisialisasi yang berbeda, solusi terbaik menghasilkan alokasi bobot saham sebagai berikut: BBCA.JK sekitar 23%, BBNI.JK sekitar 29%, BBRI.JK sekitar 21%, dan BMRI.JK sekitar 27%. Komposisi ini mencerminkan bahwa model lebih cenderung memilih BBNI.JK dan BMRI.JK memperoleh porsi relatif tinggi dibanding BBCA.JK dan BBRI.JK, yang terkait dengan keunggulan statistik return maupun profil risk-adjusted mereka di data ini. Nilai maximum utility yang negatif masih wajar karena fungsi objektif mengandung penalti variansi, kurtosis, dan biaya transaksi. Variasi hasil antar percobaan juga ditemukan yang menandakan adanya beberapa local optimum, tapi bobot portofolio terbaik tetap logis dan konsisten dengan karakter masing-masing saham yang dianalisis.

In [ ]:
print("=== PORTFOLIO DIAGNOSTICS ===")
ret_portfolio = np.dot(mu, w_best)
risk_portfolio = np.sqrt(np.dot(w_best, np.dot(Sigma, w_best)))
sharpe_portfolio = ret_portfolio / risk_portfolio if risk_portfolio > 0 else 0
turnover = np.sum(np.abs(w_best - w0))
leverage = np.sum(np.abs(w_best))

print(f"Expected Annual Return   : {ret_portfolio:.6f}  ({ret_portfolio*100:.2f}%)")
print(f"Annual Volatility (Risk): {risk_portfolio:.6f}  ({risk_portfolio*100:.2f}%)")
print(f"Sharpe Ratio            : {sharpe_portfolio:.4f}")
print(f"Turnover (vs. w₀)       : {turnover:.4f}")
print(f"Leverage (Σ|w|)         : {leverage:.4f}")

print("\n" + "-" * 65)
print("Constraint Check:")
print(f"  Budget (Σw):         {np.sum(w_best):.6f}   (harus 1.0)")
print(f"  Sharpe ratio:        {sharpe_portfolio:.4f}   (target ≥ 0.25)")
print(f"  Leverage:            {leverage:.4f}   (target ≤ 1.5)")

turnover_constraint = np.sum(np.sqrt((w_best - w0)**2 + eps))
print(f"  Turnover:            {turnover_constraint:.4f}   (target ≤ 0.6)")
print("=" * 65)


=== PORTFOLIO DIAGNOSTICS ===
Expected Annual Return   : 0.144476  (14.45%)
Annual Volatility (Risk): 0.234997  (23.50%)
Sharpe Ratio            : 0.6148
Turnover (vs. w₀)       : 0.1193
Leverage (Σ|w|)         : 1.0000

-----------------------------------------------------------------
Constraint Check:
  Budget (Σw):         1.000000   (harus 1.0)
  Sharpe ratio:        0.6148   (target ≥ 0.25)
  Leverage:            1.0000   (target ≤ 1.5)
  Turnover:            0.1193   (target ≤ 0.6)


## 3c. Interpretasi

### Tradeoff antara Risiko dan Return  
Dari hasil optimasi, diperoleh ekspektasi return tahunan portofolio sekitar 14.4%, dengan risiko atau volatilitas tahunan sebesar 23.5%. Angka Sharpe ratio yang dicapai yaitu 0.61, yang menunjukkan kinerja portofolio ini cukup baik karena efisiensi returnnya terhadap risiko jauh di atas threshold constraint yang digunakan. Dengan kata lain, kenaikan risiko yang diambil model masih sebanding dengan penambahan return yang dihasilkan, dan profil portofolio cenderung moderat.


### Pengaruh Skewness dan Kurtosis terhadap Pemilihan Saham DOminan  
Dari statistik tahunan, BBNI.JK memiliki skewness paling tinggi (0.26) dan kurtosis terendah (2.27) di antara keempat saham. Bobot terbesar juga dialokasikan ke BBNI.JK (sekitar 0.29), yang sejalan dengan teori bahwa saham dengan skewness positif (peluang untung pada extreme right tail) dan kurtosis rendah (risiko kejadian ekstrem lebih kecil) cenderung dipilih lebih dominan dalam portofolio. Sebaliknya, BMRI.JK yang kurtosisnya tinggi memperoleh porsi paling kecil dalam solusi optimal. BBCA.JK dan BBRI.JK berada di tengah-tengah, baik dari segi statistik maupun porsi bobot.

### Pengaruh Turnover  
Turnover pada model ini membatasi seberapa jauh perubahan bobot portofolio dibandingkan posisi awal agar tidak terlalu besar di setiap siklus rebalancing. Dari hasil optimasi yang saya dapat, turnovernya sekitar 0.12, jadi rata-rata perubahan porsi tiap saham relatif kecil dari kondisi awal. Pengaturan ini membuat solusi tetap responsif terhadap peluang return, risiko, dan karakteristik distribusi masing-masing saham, tetapi alokasi akhirnya tidak berubah drastis dan cenderung stabil.

